### Connect to Snowflake with Snowflake Connector

In [ ]:
# see https://github.com/Snowflake-Labs/sfguide-intro-to-snowpark-container-services/blob/main/06_docker_jupyter_service.py
# see https://github.com/Snowflake-Labs/sfguide-intro-to-snowpark-container-services/blob/main/07_docker_rest_service.py
import os, re, docker
from snowflake.connector import connect
from snowflake.core import Root

params = {
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_PASSWORD"]}
conn = connect(**params)
root = Root(conn)


### Get Snowflake repo and registry URLs

In [ ]:
schema = root.databases["test"].schemas["public"]
repo = schema.image_repositories["repo"]
repo_url = repo.fetch().repository_url
registry = re.match(r"^[^/]+", repo_url).group(0)
registry, repo_url

### Connect Docker to the Snowflake registry

In [ ]:
image_name = "service11"
client = docker.from_env()

# docker login org-acct.registry.snowflakecomputing.com -u your-username
client.login(
    username=params["user"],
    password=params["password"],
    registry=registry,
    reauth=True)

### Build the local Docker image

In [ ]:
# docker build --rm --platform linux/amd64 -t service11 .
client.images.build(path=".", platform="linux/amd64", tag=image_name)

# docker images
client.images.list()

### Test the service

In [ ]:
# docker run -d -p 8000:8000 service11
container = client.containers.run(image=image_name, detach=True, ports={8000: 8000})

os.system("curl -X GET http://localhost:8000")

### Tag image and push it into the Snowflake repo

In [ ]:
image = client.images.list()[0]
remote_image_name = f"{repo_url}/{image_name}"

# docker tag service11 org-acct.registry.snowflakecomputing.com/test/public/repo/service11
image.tag(remote_image_name, image_name)

# docker push org-acct.registry.snowflakecomputing.com/test/public/repo/service11
client.api.push(remote_image_name)

### Clean-up and show all loaded images in Snowflake repo

In [ ]:
# docker stop service11
container.stop()

images = repo.list_images_in_repository()
for image in images:
    print(image.image_name)